In [1]:
import requests
import fnmatch
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
# get the content and return a list
def getContent(url):
    # Most websites refuse GET requests from python, so we change the header to pretend we're a browser.
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    return list(soup.children)

# Search a list and return the first matching item index    
def getIndex(myList, search):
    for idx, item in enumerate(myList):
        tagData = str(myList[idx])
        if fnmatch.fnmatch(tagData, search):
            return idx

# refine by seeking a term until your soup reach the defined limit of items
def lookup(myList, desiredSize, search): 
    while len(myList) > desiredSize:
        try:
            soup = myList[getIndex(myList, search)]
            myList = list(soup.children)
        except:
            break
    
    return myList

In [3]:
# For GameSpot

def getUrl(query):
    myList = getContent(query)

    search = '*editorial river search-results*'
    tempList = lookup(myList, 2, search)
    tempList = lookup(tempList, 1, search)
    url = str(tempList).split('>')[3]
    url = url.split('"')
    return 'https://www.gamespot.com' + url[1]

# Build and get a list of search queries
def get_gs_sources(dataFrame):
    url = []
    sQuery = 'https://www.gamespot.com/search/?q='
    search = ''
    counter = 0

    for game in df['Name']:
        search = game.replace(' ','+')
        url.append(getUrl(sQuery + search))
        counter += 1
        if counter % 10 == 0:
            print(counter)
        #queries.append(sQuery + search)
    
    return url

def scrap_gs(urls):
    counter = 0
    games = []
    year = []
    Rscores = []
    Mscores = []
    Uscores = []
    
    for url in urls:
        myList = getContent(url)
        
        #Get Title
        search = '*title*'
        tempList = lookup(myList, 2, search)

        title = str(tempList).split("'")[1]
        title = title.replace(' - GameSpot','')
        games.append(title)
        
        # Get Release Year
        search = '*First Released*'
        tempList = lookup(myList, 2, search)

        if (tempList != myList):
            releaseY = str(tempList[1]).split(', ')
            releaseY = str(releaseY).replace('<span>','')
            releaseY = releaseY.replace('</span>','')           
            year.append(releaseY)
        else:
            year.append(None)        

        # Get Review Score
        search = '*Review score*'
        temp = lookup(myList, 1, search)

        if (temp != myList):
            reviewScore = str(temp).split('>')
            reviewScore = reviewScore[3].split('<')
            Rscores.append(reviewScore[0])
        else:
            Rscores.append(None)

        # Get Metacritic Score
        search = '*Metascore*'
        tempList = lookup(myList, 2, search)

        if (tempList != myList):
            metaScore = str(tempList[0]).split('blank">')
            metaScore = metaScore[1].split('<')
            Mscores.append(metaScore[0])
        else:
            Mscores.append(None)

        # Get User Score
        search = '*userAvg*'
        tempList = lookup(myList, 2, search)

        if (tempList != myList):
            userScore = str(tempList).split('\n')
            userScore = userScore[2].replace(' ','')
            Uscores.append(userScore)
        else:
            Uscores.append(None)    
        
        # progress track
        counter += 1
        if counter % 10 == 0:
            print(counter)  
            
    return games, year, Rscores, Mscores, Uscores

In [4]:
# For IGN

def get_ign_sources(df):
    source = []
    counter = 0
    for name in df['Name']:
        url = 'https://ca.ign.com/search?q=' + name.replace(' ','%20')
        myList = getContent(url)

        # Get source
        search = '*search-item-title*'
        tempList = lookup(myList, 2, search)[1]
        tempList = str(tempList).split('"')[1]
        source.append(tempList)
        
        # progress track
        counter += 1
        if counter % 10 == 0:
            print(counter)
        
    return source


def scrap_ign(urls):
    counter = 0
    game = []
    year = []
    score = []
    
    for url in urls:
        myList = getContent(url)

        # Get title
        search = '*title*'
        tempList = lookup(myList, 1, search)
        if (tempList != myList):
            tempList = str(tempList).split(' - ')[0][2:]
            game.append(str(tempList))
        else:
            game.append(None)

        # Get score
        search = '*ratingValue*'
        tempList = lookup(myList, 1, search)
        if (tempList != myList):
            tempList = str(tempList).split(',')
            if len(tempList) > 4:            
                tempList = tempList[-3].split(':')[1]
            else:
                tempList = str(tempList)[9:20].replace(' ','')
            score.append(str(tempList))
        else:
            score.append(None)

        # Get year
        search = '*Released</dt><dd class=*'
        tempList = lookup(myList, 1, search)
        if (tempList != myList):
            # give it a push and search again
            tempList = lookup(list(tempList[0].children), 1, search)
            tempList = str(tempList).split(',')[2]
            tempList = tempList.split('<')[0].replace(' ','')
            year.append(tempList)
        else:
            year.append(None)

        # progress track
        counter += 1
        if counter % 10 == 0:
            print(counter)
            
    return game, year, score

In [5]:
print('reading game titles from source')
df = pd.read_csv('Video_Games_Sales_2016.csv')

print('\n getting urls for scraping IGN:')
url = get_ign_sources(df)

print('\n collecting contents from IGN:')
g, y, s = scrap_ign(url)

reading game titles from source

 getting urls for scraping IGN:
10
20
30
40
50
60
70
80
90

 collecting contents from IGN:
10
20
30
40
50
60
70
80
90


In [7]:
df['IGN_title'] = g
df['IGN_year'] = y
df['IGN_score'] = s
df['IGN_source'] = url

df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,IGN_title,IGN_year,IGN_score,IGN_source
0,Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E,Wii Sports,2006,7.5,https://ca.ign.com/games/wii-sports/wii-826987
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,The Super Mario Bros Super Show,2006,6,https://ca.ign.com/tv/the-super-mario-bros-sup...
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,Mario Kart Wii,2008,8.9,https://ca.ign.com/games/mario-kart-wii/wii-94...
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E,Wii Sports Resort Video Review,None,None,https://ca.ign.com/videos/2009/07/16/wii-sport...
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,Pokemon,2000,None,https://ca.ign.com/tv/pokemon


In [8]:
print('\n getting urls for scraping gs:')
url = get_gs_sources(df)

print('\n collecting contents from gs:')
g, y, r, m, u = scrap_gs(url)


 getting urls for scraping gs:
10
20
30
40
50
60
70
80
90

 collecting contents from gs:
10
20
30
40
50
60
70
80
90


In [9]:
df['GS_title'] = g
df['GS_year'] = y
df['GS_review'] = r
df['GS_metacritic'] = m
df['GS_user'] = u
df['GS_source'] = url

df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,IGN_title,IGN_year,IGN_score,IGN_source,GS_title,GS_year,GS_review,GS_metacritic,GS_user,GS_source
0,Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,...,Wii Sports,2006,7.5,https://ca.ign.com/games/wii-sports/wii-826987,Wii Sports,"['Nov 19', '2006']",7.8,76,8.1,https://www.gamespot.com/wii-sports/
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,...,The Super Mario Bros Super Show,2006,6,https://ca.ign.com/tv/the-super-mario-bros-sup...,Super Mario Bros.,"['Oct 18', '1985']",6.8,None,8.9,https://www.gamespot.com/classic-nes-series-su...
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,Mario Kart Wii,2008,8.9,https://ca.ign.com/games/mario-kart-wii/wii-94...,Mario Kart Wii,"['Apr 27', '2008']",8.5,82,8.7,https://www.gamespot.com/mario-kart-wii/
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,...,Wii Sports Resort Video Review,None,None,https://ca.ign.com/videos/2009/07/16/wii-sport...,Wii Sports Resort,"['Jul 26', '2009']",8,80,8.3,https://www.gamespot.com/wii-sports-resort/
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,...,Pokemon,2000,None,https://ca.ign.com/tv/pokemon,Pokemon GO,"['Jul 6', '2016']",7,69,6.1,https://www.gamespot.com/pokemon-go/


In [ ]:
# get content and refine list to 3 items
#url = 'https://www.gamespot.com/new-super-mario-bros/'
#url = 'https://www.gamespot.com/wii-fit/'
#url = 'https://www.gamespot.com/classic-nes-series-super-mario-bros/'
#url = 'https://www.gamespot.com/tetris/'

In [33]:
def clean_text(raw_text, char_list):
    clean_text = str(raw_text)
    for c in char_list :
        clean_text = clean_text.replace(c, '')
    return  clean_text.lower()   

def clean_list(df, score, title, year):
    char_list = ['.', ':', '-', '!', '/', '\\']
    remove_list = []

    for idx, e in enumerate(df['Name']):
        remove = False
        
        if df.loc[idx, score] != None:
            og_name = clean_text(e, char_list)
            scrap_name = clean_text(df.loc[idx, title],char_list)
            if og_name != scrap_name:
                if df.loc[idx, 'Year_of_Release'] != df.loc[idx, year]:
                    remove = True
        else:
            remove = True
        if remove:
            remove_list.append(idx)
    
    return remove_list

In [39]:
# adjust gs date
for i, date in enumerate(df['GS_year']):
    test = date
    test = clean_text(test, ["[", "]", "'"])
    test = test.split(' ')
    df.loc[i, 'GS_year'] = test[-1]

df['GS_year'].head()

0    2006
1    1985
2    2008
3    2009
4    2016
Name: GS_year, dtype: object

In [45]:
# Clean IGN
remove_list = clean_list(df, 'IGN_score', 'IGN_title', 'IGN_year')

print(len(remove_list), ' rows of IGN data removed')

for i in remove_list:
    df.loc[i, 'IGN_title'] = None
    df.loc[i, 'IGN_year'] = None
    df.loc[i, 'IGN_score'] = None
    df.loc[i, 'IGN_source'] = None

# Clean GS
remove_list = clean_list(df, 'GS_user', 'GS_title', 'GS_year')

print(len(remove_list), ' rows of GS data removed')

for i in remove_list:
    df.loc[i, 'GS_title'] = None
    df.loc[i, 'GS_year'] = None
    df.loc[i, 'GS_review'] = None
    df.loc[i, 'GS_user'] = None
    df.loc[i, 'GS_metacritic'] = None
    df.loc[i, 'GS_source'] = None

48  rows of IGN data removed
19  rows of GS data removed


In [53]:
df.to_csv('new.csv',  index=False)